# 주차어때 데이터 파트


## 1. 주차장 정보에서 url 가져오기

### 1-1 데이터 정제

In [4]:
import pandas as pd
import re             # 정규표현식

In [5]:
data = pd.read_csv("서울시 공영주차장 안내 정보.csv", encoding='utf-8')

In [6]:
columns = ['주차장코드','주차장명', '주소'] #, '주차장 위치 좌표 위도', '주차장 위치 좌표 경도']
data2 = data[columns].copy()
data2.head(3)

,주차장코드,주차장명,주소
0,1010089,초안산근린공원주차장(구),도봉구 창동 24-0
1,1012254,마들스타디움(근린공원)(구),노원구 상계동 770-2
2,1013181,마장동(건물) 공영주차장(구),성동구 마장동 463-2


In [7]:
data['주차장코드'].value_counts()

1538796    310
1503518    299
1495387    244
1552996    194
1536637    175
          ... 
1579833      1
1579832      1
1579598      1
1579566      1
984688       1
Name: 주차장코드, Length: 1107, dtype: int64

데이터에 중복된 column이 존재한다. 중복된 주차장코드의 column들이 존재하는 이유는 다음과 같다.

> 앞 부분만 보시면 동일한 데이터로 보일 수 있으나, 시트 가장 오른쪽의 위도/경도 컬럼을 확인해 보시면 값이 다르게 표시되고 있습니다.
> 노상주차장의 경우 도로의 한쪽을 주차구역으로 만든 것으로, 주차장을 설치한 도로 길이만큼 위도/경도값이 여럿일 수 밖에 없습니다.
> 위와 같은 이유로 시작점이나 중간점, 끝점을 하나 지정하셔서 대표값으로 사용하시면 될 것으로 보입니다.
> -서울시 공영주차장 안내 정보 관리자

리뷰는 그룹화하여 제공되므로, 그룹 별로 시작값만 남기고 제거한다. 중복이 사라져 많이 비므로 인덱스를 재설정한다. 중복 제거 전 16000개의 데이터가 있었는데 제거 후 1107개가 남는다.

In [8]:
data2 = data2.drop_duplicates(['주차장코드'])
data2 = data2.reset_index(drop=True)
data2[50:55]

,주차장코드,주차장명,주소
50,1235916,청계2(남) 공영주차장(시),중구 입정동 29-1
51,1236307,청계1(남) 공영주차장(시),중구 수표동 65-4
52,1236612,남대문 화물 공영주차장(시),중구 남대문로4가 24-1
53,1236742,중앙일보사옆 관광버스전용 주차장(시),중구 순화동 7-0
54,1236949,한진면세점앞 노상 공영주차장(구),중구 남대문로5가 533-0 0


## 1-2 검색어 생성

주차장명을 그대로 검색하면 주차장이 안 나오는 경우가 있다.
다음과 같은 원칙에 따라 주차장명을 가공해 검색어를 새로 만든다.
* 괄호 안에 들어가는 말은 지운다.
* '주차장' 이라는 단어가 포함되어 있지 않은 경우 끝에 붙인다.

예시: '은평평화공원(구)' ---> '은평평화공원 주차장'

In [10]:
def rename_parking_lot(old_name):
    new_name = re.sub(r'\([^)]*\)', '', old_name) # 괄호 안에 든 문자열 제거
    if '주차장' not in new_name:
        new_name = new_name + " 주차장"          # 주차장 이라는 문자열 없으면 붙여줌 
    return new_name

In [11]:
data2['검색어'] = data2['주차장명'].map(rename_parking_lot)
data2.head(5)

,주차장코드,주차장명,주소,검색어
0,1010089,초안산근린공원주차장(구),도봉구 창동 24-0,초안산근린공원주차장
1,1012254,마들스타디움(근린공원)(구),노원구 상계동 770-2,마들스타디움 주차장
2,1013181,마장동(건물) 공영주차장(구),성동구 마장동 463-2,마장동 공영주차장
3,1025695,영등포여고 공영(구),영등포구 신길동 184-3,영등포여고 공영 주차장
4,1025696,당산근린공원 공영(구),영등포구 당산동3가 385-0,당산근린공원 공영 주차장


### 1-3 URL 저장
URL 이라는 이름의 열을 만들어, '검색어' 열에 대한 검색 결과를 보여주는 URL을 저장한다.

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import time

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [13]:
data2['url'] = ''
data2.head(2)

,주차장코드,주차장명,주소,검색어,url
0,1010089,초안산근린공원주차장(구),도봉구 창동 24-0,초안산근린공원주차장,
1,1012254,마들스타디움(근린공원)(구),노원구 상계동 770-2,마들스타디움 주차장,


url 저장 전에 해당 열을 공백으로 초기화해준다.

In [14]:
driver = set_chrome_driver()

In [19]:
# 77m

for i, keyword in enumerate(data2['검색어'].tolist()):
    if data2['url'][i] == '':
        try:
            naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'
            # 검색 url
            driver.get(naver_map_search_url)
            time.sleep(4) # 대기
            cu = driver.current_url
            res_code = re.findall(r"place/(\d+)", cu) # 번호 알아냄
            final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'
            data2['url'][i] = final_url
        except IndexError:
            data2['url'][i] = ''
        
data2

C:\Users\00_ma\AppData\Local\Temp\ipykernel_25020\2398377516.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['url'][i] = final_url
C:\Users\00_ma\AppData\Local\Temp\ipykernel_25020\2398377516.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['url'][i] = ''


,주차장코드,주차장명,주소,검색어,url
0,1010089,초안산근린공원주차장(구),도봉구 창동 24-0,초안산근린공원주차장,https://pcmap.place.naver.com/restaurant/35824...
1,1012254,마들스타디움(근린공원)(구),노원구 상계동 770-2,마들스타디움 주차장,https://pcmap.place.naver.com/restaurant/10357...
2,1013181,마장동(건물) 공영주차장(구),성동구 마장동 463-2,마장동 공영주차장,https://pcmap.place.naver.com/restaurant/70874...
3,1025695,영등포여고 공영(구),영등포구 신길동 184-3,영등포여고 공영 주차장,https://pcmap.place.naver.com/restaurant/36005...
4,1025696,당산근린공원 공영(구),영등포구 당산동3가 385-0,당산근린공원 공영 주차장,https://pcmap.place.naver.com/restaurant/37416...
...,...,...,...,...,...
1102,968503,삼양마을공원지하 공영주차장(구),강북구 미아동 748-2,삼양마을공원지하 공영주차장,
1103,968514,색동공원 공영주차장(구),강북구 수유동 49-7,색동공원 공영주차장,https://pcmap.place.naver.com/restaurant/36010...
1104,980509,비봉주차장(구),종로구 구기동 139-9,비봉주차장,https://pcmap.place.naver.com/restaurant/37304...
1105,984617,신도림역환승주차장(구),구로구 구로동 1-4,신도림역환승주차장,https://pcmap.place.naver.com/restaurant/19491...


In [20]:
data2

,주차장코드,주차장명,주소,검색어,url
0,1010089,초안산근린공원주차장(구),도봉구 창동 24-0,초안산근린공원주차장,https://pcmap.place.naver.com/restaurant/35824...
1,1012254,마들스타디움(근린공원)(구),노원구 상계동 770-2,마들스타디움 주차장,https://pcmap.place.naver.com/restaurant/10357...
2,1013181,마장동(건물) 공영주차장(구),성동구 마장동 463-2,마장동 공영주차장,https://pcmap.place.naver.com/restaurant/70874...
3,1025695,영등포여고 공영(구),영등포구 신길동 184-3,영등포여고 공영 주차장,https://pcmap.place.naver.com/restaurant/36005...
4,1025696,당산근린공원 공영(구),영등포구 당산동3가 385-0,당산근린공원 공영 주차장,https://pcmap.place.naver.com/restaurant/37416...
...,...,...,...,...,...
1102,968503,삼양마을공원지하 공영주차장(구),강북구 미아동 748-2,삼양마을공원지하 공영주차장,
1103,968514,색동공원 공영주차장(구),강북구 수유동 49-7,색동공원 공영주차장,https://pcmap.place.naver.com/restaurant/36010...
1104,980509,비봉주차장(구),종로구 구기동 139-9,비봉주차장,https://pcmap.place.naver.com/restaurant/37304...
1105,984617,신도림역환승주차장(구),구로구 구로동 1-4,신도림역환승주차장,https://pcmap.place.naver.com/restaurant/19491...


In [21]:
data2.to_csv('url_completed_in_progress.csv', encoding = 'utf-8-sig')

완성한 데이터프레임을 csv 파일로 저장한다. 

## 2. URL 별 리뷰 크롤링

1. 에서 추출한 URL을 바탕으로 크롤링을 진행한다.

In [1]:
import pandas as pd
import time
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import InvalidArgumentException

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [12]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

driver = webdriver.Chrome(service=Service('chromedriver.exe'), options=chrome_options)



In [3]:
df = pd.read_csv('url_completed_in_progress.csv')

In [4]:
df['rank'] = 0

주차장 하나에 해당하는 리뷰들을 하나의 csv 파일로 저장한다.

평균별점 같은 경우에 csv 파일에 덧붙인다.

In [5]:
result_df = pd.DataFrame()
result_df['parkingId'] = df['주차장코드']
result_df['url'] = df['url']
result_df['rank'] = 0
result_df.dropna(axis=0, inplace=True)
result_df.reset_index(drop=True, inplace=True)

result_df

,parkingId,url,rank
0,1010089,https://pcmap.place.naver.com/restaurant/35824...,0
1,1012254,https://pcmap.place.naver.com/restaurant/10357...,0
2,1013181,https://pcmap.place.naver.com/restaurant/70874...,0
3,1025695,https://pcmap.place.naver.com/restaurant/36005...,0
4,1025696,https://pcmap.place.naver.com/restaurant/37416...,0
...,...,...,...
676,968483,https://pcmap.place.naver.com/restaurant/21185...,0
677,968494,https://pcmap.place.naver.com/restaurant/21185...,0
678,968514,https://pcmap.place.naver.com/restaurant/36010...,0
679,980509,https://pcmap.place.naver.com/restaurant/37304...,0


절차 간소화를 위해 해당 장소의 별점만 먼저 수집한다.

In [6]:
# 별점만 - 33m
for i in tqdm(range(result_df.shape[0])):
        try:
            driver.get(result_df['url'][i])
            thisurl = result_df['url'][i]
            time.sleep(2)
        except InvalidArgumentException:
            continue
        
        try:
            rank = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text
            result_df['rank'][i] = int(float(rank))    # 평균 별점
        except:
            pass

  0%|          | 0/681 [00:00<?, ?it/s]C:\Users\00_ma\AppData\Local\Temp\ipykernel_9496\838816152.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['rank'][i] = int(float(rank))    # 평균 별점
100%|██████████| 681/681 [32:47<00:00,  2.89s/it]


In [7]:
result_df.to_csv("result_df_tmp.csv")

In [8]:
result_df

,parkingId,url,rank
0,1010089,https://pcmap.place.naver.com/restaurant/35824...,4
1,1012254,https://pcmap.place.naver.com/restaurant/10357...,0
2,1013181,https://pcmap.place.naver.com/restaurant/70874...,4
3,1025695,https://pcmap.place.naver.com/restaurant/36005...,0
4,1025696,https://pcmap.place.naver.com/restaurant/37416...,4
...,...,...,...
676,968483,https://pcmap.place.naver.com/restaurant/21185...,4
677,968494,https://pcmap.place.naver.com/restaurant/21185...,0
678,968514,https://pcmap.place.naver.com/restaurant/36010...,0
679,980509,https://pcmap.place.naver.com/restaurant/37304...,4


전체 리뷰에 대해 수집한다.

In [14]:
# 리뷰 전체
# 1h15m

import os.path
from tqdm import tqdm

for i in tqdm(range(result_df.shape[0])):
    if os.path.exists("reviews/" + str(result_df['parkingId'][i])):
        # 이미 크롤링 한 거 패쓰
        continue
    try:
        driver.get(result_df['url'][i])
        thisurl = result_df['url'][i]
        time.sleep(2)
    except InvalidArgumentException:
        continue
    
    # 더보기 버튼 다 누르기 
    while True:
        try:
            time.sleep(1)
            driver.find_element(by=By.TAG_NAME, value='body').send_keys(Keys.END)
            time.sleep(3)
            driver.find_element(by=By.CSS_SELECTOR, value='#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.lfH3O > a').click()
            
            time.sleep(3)
            driver.find_element(by=By.TAG_NAME, value='body').send_keys(Keys.END)
            time.sleep(1)
        except NoSuchElementException:
            # print("-더보기버튼 모두 클릭 완료-")
            break
    # 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    comments = []
    for comment in soup.find_all('span', class_='zPfVt'):
        comments.append(comment.get_text())    # 리뷰들 
    
    # 리뷰를 모아서 csv 파일로 저장
    df2 = pd.DataFrame({"Comments":comments})
    file_name = "reviews/" + str(result_df['parkingId'][i]) + ".csv"
    df2.to_csv(file_name, index= False, encoding='utf-8', escapechar="\\") #파일경로
        

100%|██████████| 681/681 [11:46:10<00:00, 62.22s/it]   


In [15]:
driver.quit()    # 드라이버 종료

In [16]:
result_df.to_csv('result_df.csv', encoding='utf-8')

## 3. 리뷰 데이터 전처리 

In [33]:
#import할 패키지 목록
import re
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
from hanspell import spell_checker
from collections import Counter
import numpy as np
import os

DB의 Table에 들어가는 최종 결과 csv를 만든다.

DB의 스키마는 다음과 같다.
- review_id BIGINT
- created_date DATETIME(6)
- modified_date DATETIME(6)
- eval_costefficient VARCHAR(255)
- eval_parkinglevel VARCHAR(255)
- eval_revisit VARCHAR(255)
- eval_space VARCHAR(255)
- eval_staff VARCHAR(255)
- star_score INT
- parking_code VARCHAR(100)
- user_id BIGINT

In [34]:
headers = ['review_id', 'created_date', 'modified_date', 'eval_costefficient', 'eval_parkinglevel', 'eval_revisit', 'eval_space', 'eval_staff', 'star_score', 'parking_code', 'user_id']
result = pd.DataFrame()


In [ ]:
result_df = pd.read_csv("result_df.csv")
original_df = pd.read_csv("url_completed_in_progress.csv")

In [70]:
# result는 하나의 리뷰 단위로 들어가는 칼럼이라 그냥 큰 반복문에서 돌려야겠다.. 
result = result_df[['parkingId', 'rank']].copy()
result.columns = ['parking_code', 'star_score']

# 크롤링 해서 가져온 review_id는 PK라서 10000에서 시작해서 증가해주는걸로 해줬다.
result['review_id'] = 10000
# 크롤링 해서 가져온 user_id는 일괄적으로 1(이름:superuser)로 설정
result['user_id'] = 1

result['eval_space'] = "MIDDLE"
result['eval_parkinglevel'] = "FINE"
result['eval_costefficient'] = "FAIR"
result['eval_staff'] = "MODERATE"
result['eval_revisit'] = "YES"
print(result.head())
print(result.shape)

   parking_code  star_score  review_id  user_id eval_space eval_parkinglevel  \
0       1010089           4      10000        1     MIDDLE              FINE   
1       1012254           0      10000        1     MIDDLE              FINE   
2       1013181           4      10000        1     MIDDLE              FINE   
3       1025695           0      10000        1     MIDDLE              FINE   
4       1025696           4      10000        1     MIDDLE              FINE   

  eval_costefficient eval_staff eval_revisit  
0               FAIR   MODERATE          YES  
1               FAIR   MODERATE          YES  
2               FAIR   MODERATE          YES  
3               FAIR   MODERATE          YES  
4               FAIR   MODERATE          YES  
(681, 9)


데이터프레임의 형태가 잡혔으니 키워드 추출을 수행한다.

In [50]:
def extract_word(text):
    hangul = re.compile('[^가-힣]')
    result = hangul.sub(' ', text)
    return result

In [51]:
def spell_check(text):
    if text:
        spelled_sent = spell_checker.check(text)
        hanspell_sent = spelled_sent.checked
        return hanspell_sent
    else:
        return ""

In [52]:
keywords = [["넓다"], ["좁다", "협소"], \
            ["쉽다", "편하다"], ["어렵다", "힘들다"], \
            ["저렴하다", "싸다", "쌈"], ["비싸다"], \
            ["친절하다"], ["불친절하다"], \
            ["좋다", "굿", "굳", "자주", "추천"], ["아쉽다", "별로"]]

"""
0: evalSpace, 1: evalParkinglevel, 2: evalCostefficient, 
3: evalStaff, 4: evalRevisit
"""

'\n0: evalSpace, 1: evalParkinglevel, 2: evalCostefficient, \n3: evalStaff, 4: evalRevisit\n'

In [53]:
"""
긍정 키워드와 부정 키워드의 비율을 놓고, 
어느 한쪽의 비율이 2:1 보다 클 때만 해당 키워드를 출력한다.
"""

def parse_keyword(keywords_freq):
    keywords_freq_ratio = np.array(keywords_freq).reshape(5, 2)
    keywords_result = np.arange(15).reshape(5, 3)
    for i in range(5):
        if keywords_freq_ratio[i][0] + keywords_freq_ratio[i][1] == 0:
            ratio = 1
        elif keywords_freq_ratio[i][0] == 0:
            ratio = 0
        elif keywords_freq_ratio[i][1] == 0:
            ratio = 999
        else:
            ratio = keywords_freq_ratio[i][0] / keywords_freq_ratio[i][1]
        if ratio >= 2:
            # 좋음!
            keywords_result[i] = [1, 0, 0]
        elif ratio <= 0.5:
            # 나쁨!
            keywords_result[i] = [0, 0, 1]
        else:
            # 중간 or 정보 없음
            keywords_result[i] = [0, 1, 0]

    return keywords_result
        

In [54]:
def get_keyword_result(filepath):
    review = pd.read_csv(filepath, encoding='utf-8')
    
    review['Comments'] = review['Comments'].apply(lambda x:extract_word(x))
    review['Comments'] = review['Comments'].apply(lambda x:spell_check(x)) 
    # 형태소분석 - Okt
    okt = Okt()
    words = " ".join(review['Comments'].tolist())
    words = okt.morphs(words, stem=True)
    # 불용어
    with open('stopwords.txt', 'r') as f:
        list_file = f.readlines()
    stopwords = list_file[0].split(",")
    remove_stopwords = [x for x in words if x not in stopwords]
    # (단어, 빈도수) 쌍 추출
    frequent = Counter(remove_stopwords).most_common()
    
    # 우리 프로젝트에 필요한 키워드와 대조.
    keywords_freq = [0] * 10
    # 키워드와 비교하여 Count
    frequent = Counter(remove_stopwords).most_common()
    for morp in frequent:
        for i in range(len(keywords)):
            for j in range(len(keywords[i])):
                if morp[0] == keywords[i][j]:
                    keywords_freq[i] += morp[1]
    keyword_result = parse_keyword(keywords_freq)
    return keyword_result


In [65]:
# print(str(result['parking_code'][0]))
# result.shape[0]
print(result.at[result.index[0], 'review_id'])

10000


In [71]:
import os

# 26s

review_id = 10001
dir_name = "reviews/preprocess/"
for i in tqdm(range(0, result.shape[0])):
    result.at[result.index[i], 'review_id'] = review_id
    filepath = dir_name + str(result['parking_code'][i]) + ".csv"
    keyword_result = get_keyword_result(filepath)
    
    # print(filepath)
    for j in range(5):
        good = keyword_result[j][0]
        mid = keyword_result[j][1]
        bad = keyword_result[j][2]
        # print(j, good, mid, bad)
        
        if j == 0:
            if good == 1:
                result.at[result.index[i], 'eval_space'] = "LARGE"
            elif mid == 1:
                result.at[result.index[i], 'eval_space'] = "MIDDLE"
            else:
                result.at[result.index[i], 'eval_space'] = "SMALL"
        elif j == 1:
            if good == 1:
                result.at[result.index[i], 'eval_parkinglevel'] = "EASY"
            elif mid == 1:
                result.at[result.index[i], 'eval_parkinglevel'] = "FINE"
            else:
                result.at[result.index[i], 'eval_parkinglevel'] = "HARD"
        elif j == 2:
            if good == 1:
                result.at[result.index[i], 'eval_costefficient'] = "CHEAP"
            elif mid == 1:
                result.at[result.index[i], 'eval_costefficient'] = "FAIR"
            else:
                result.at[result.index[i], 'eval_costefficient'] = "OVERPRICED"
        elif j == 3:
            if good == 1:
                result.at[result.index[i], 'eval_staff'] = "FRIENDLY"
            elif mid == 1:
                result.at[result.index[i], 'eval_staff'] = "MODERATE"
            else:
                result.at[result.index[i], 'eval_staff'] = "COLD"
        elif j == 4:
            if good == 1 or mid == 1:
                result.at[result.index[i], 'eval_revisit'] = "YES"
            else:
                result.at[result.index[i], 'eval_revisit'] = "NO"
    
    review_id += 1


100%|██████████| 681/681 [00:26<00:00, 25.96it/s]


In [72]:
print(result)
result.to_csv("real_result.csv", index=False)

     parking_code  star_score  review_id  user_id eval_space  \
0         1010089           4      10001        1      LARGE   
1         1012254           0      10002        1     MIDDLE   
2         1013181           4      10003        1      SMALL   
3         1025695           0      10004        1     MIDDLE   
4         1025696           4      10005        1      LARGE   
..            ...         ...        ...      ...        ...   
676        968483           4      10677        1      SMALL   
677        968494           0      10678        1     MIDDLE   
678        968514           0      10679        1     MIDDLE   
679        980509           4      10680        1     MIDDLE   
680        984617           4      10681        1     MIDDLE   

    eval_parkinglevel eval_costefficient eval_staff eval_revisit  
0                EASY              CHEAP   MODERATE          YES  
1                EASY              CHEAP   MODERATE          YES  
2                EASY         

(5,2) 형태로 어떤 keyword인지 도출되었다.